In [88]:
from tensorflow import keras


In [2]:
Model_path = "C:/Users/uqcche32/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/Single_Head_Residual_attention_realForm64_64Local"
#Model_path = "E:/learning resource/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/"

In [3]:
from CustomLayers import *
from GS_composer import *
from Functions import *
from ClassModel import *

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [4]:
#

In [5]:
model_folder = "pachy_MultiHeadAttentionLNN_1"
#model_folder = "test_model"
full_path = Model_path + "/" + model_folder
model = keras.models.load_model(full_path,custom_objects={"MultiHead_QKV_BlockAttention": MultiHead_QKV_BlockAttention,
                                                         "SNPBlockLayer":SNPBlockLayer})

In [44]:
model.summary()
#model = keras.Model(inputs=input1, outputs=QV_output)
model.compile(optimizer="RMSprop", loss="mean_squared_error")

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_1 (InputLayer)      [(None, 26086, 1)]   0                                            
__________________________________________________________________________________________________
zero_padding1d (ZeroPadding1D)  (None, 26086, 1)     0           input_layer_1[0][0]              
__________________________________________________________________________________________________
locally_connected1d (LocallyCon (None, 2608, 64)     1669120     zero_padding1d[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 2608, 64)     4160        locally_connected1d[0][0]        
______________________________________________________________________________________________

In [33]:
#!pip install shap
import shap

In [34]:
import numpy as np
import pandas as pd
import os
mother_path = "C:/Users/uqcche32/OneDrive - The University of Queensland/"

In [35]:
class p_args:
    
    def __init__(self):
        
        self.ped = mother_path + "PhD/data/sugarcane_disease/sugarcane_disease"
        self.pheno = mother_path + "PhD/data/sugarcane_disease/sugarcane_disease.phen"
        self.mpheno = 1
        self.index = mother_path + "PhD/data/sugarcane_disease/sugarcane_disease.index"
        self.model = "MultiHead Attention LNN"
        self.annotation = None# "E:/learning resource/OneDrive - The University of Queensland/PhD/data/sugarcane_disease/subset/disease_subset.anno"
        self.trait = "smut"
        self.output = "./Attention_Test_win"
        
        self.load = None
        self.save = True
        self.plot = True
        
        self.round = 1
        self.epoch = 10
        self.rank =False
        self.lr = 0.0001
        self.batch = 32
        self.mean = False
        self.width = 64
        self.depth = 2
        self.quiet = 1
        self.num_heads = 1
        self.residual = True
        self.embedding = 12
        
        
args = p_args()

In [73]:
config = None
if args.output[0] == "/":
    locat = '/' + args.output.strip('/') + '/'
else:
    locat = args.output.strip('/') + '/'
if not os.path.exists(locat):
    os.mkdir(locat)
composer = ML_composer()
composer._model
composer.silence_mode = 2
composer.args = args
composer.get_data(config,args=args)

1    939
3    940
4    940
2    940
5    943
Name: 2, dtype: int64
Get genotype shape: (4702, 26086)
   7   8   9   10  11  12  13  14  15
1   0   2   0   1   2   1   0   1   2
2   1   2   1   1   2   1   0   1   1
3   1   2   0   0   1   0   1   1   2
4   1   2   0   0   1   0   1   1   2
5   0   2   1   1   2   0   2   1   1
6   0   2   0   1   2   1   0   1   2
7   1   2   1   1   2   1   2   1   2
8   0   2   0   1   1   0   1   0   2
9   1   2   1   1   2   1   2   1   2
Running data check
GENO
Samples are not aligned with same order
FAM
PHENO
Samples are not aligned with same order
INDEX
Samples are not aligned with same order


In [74]:

composer.prepare_model()
index_ref = composer.prepare_cross_validate()
train_idx,valid_idx = index_ref[0]
composer.prepare_training(train_idx,valid_idx)

Overall population: 4702
759 individuals need to be removed due to the miss phenotype
Filtered population: 3943
Mean of train phenotype: 4.715129635707253
Use raw phenotype as the target
16    1.0
39    3.0
44    3.0
47    2.0
92    3.0
Name: 2, dtype: float64


In [75]:
features_train,target_train = composer._model["INIT_MODEL"].data_transform(composer.train_data,composer.train_pheno, pheno_standard = composer.args.rank)
features_val,target_val = composer._model["INIT_MODEL"].data_transform(composer.valid_data,composer.valid_pheno, pheno_standard = composer.args.rank)

USE Attention CNN MODEL as training method
Convert data to np.array float32
The transformed SNP shape: (3047, 26086, 1)
USE Attention CNN MODEL as training method
Convert data to np.array float32
The transformed SNP shape: (896, 26086, 1)


In [84]:

background = features_train[np.random.choice(features_train.shape[0], 2, replace=False)]
print(model.predict(background))
e = shap.DeepExplainer(model, background)

[[5.0258956]
 [4.6213245]]


In [90]:

shap_values = e.shap_values(features_val[:1])

# plot the feature attributions
shap.image_plot(shap_values, -target_train[:1])

ValueError: tf.enable_eager_execution must be called at program startup.

In [71]:
keras.backend.clear_session()
gc.collect()

211

In [82]:
features_val[:1].shape

(1, 26086, 1)